In [11]:
import pyrosm
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon


In [7]:
# Load places data from CSV
places_df = pd.read_csv('Annex A.csv')


In [8]:
def load_osm_graph(pbf_file, mode='drive'):
    # Initialize the reader for the .pbf file
    osm = pyrosm.OSM(pbf_file)
    
    # Extract the network data (by default for 'driving')
    if mode == 'drive':
        network = osm.get_network(network_type='driving')
    elif mode == 'walk':
        network = osm.get_network(network_type='walking')
    elif mode == 'bike':
        network = osm.get_network(network_type='cycling')
    else:
        raise ValueError(f"Mode '{mode}' is not supported. Use 'drive', 'walk', or 'bike'.")
    
    # Convert the network to a NetworkX graph for routing
    G = ox.graph_from_gdfs(network[['u', 'v', 'geometry']], edges=network)
    
    return G

In [9]:
# Function to generate isochrone polygon for a given location
def generate_isochrone(osm_graph, lat, lon, travel_time=15, mode='drive'):
    # Get the nearest network node to the place
    node = ox.distance.nearest_nodes(osm_graph, lon, lat)

    # Generate an isochrone based on travel time (15 minutes as an example)
    trip_time = travel_time * 60  # Convert minutes to seconds

    # Get subgraph limited by travel time
    subgraph = ox.utils_graph.truncate_graph_by_time(osm_graph, node, trip_time, weight='travel_time')
    isochrone_polygon = ox.utils_graph.graph_to_gdfs(subgraph, nodes=False, edges=False)['geometry'].unary_union

    return isochrone_polygon

In [12]:
osm_pbf_file = 'C:/Users/jake/geocoding_data/britain-and-ireland-latest.osm.pbf'
osm_graph = load_osm_graph(osm_pbf_file, mode='drive')

In [ ]:
# List to store isochrones for each place
isochrones = []

# Iterate over each place and generate isochrone
for index, row in places_df.iterrows():
    name = row['Permanent']
    lat = row['LATITUDE']
    lon = row['LONGITUDE']
    
    # Generate isochrone for each place
    polygon = generate_isochrone(osm_graph, lat, lon)
    
    # Store as a GeoDataFrame with associated name
    isochrones.append(gpd.GeoDataFrame({'Name': [name], 'geometry': [polygon]}, crs="EPSG:4326"))

# Combine all isochrones into a single GeoDataFrame
isochrones_gdf = gpd.GeoDataFrame(pd.concat(isochrones, ignore_index=True), crs="EPSG:4326")

# Export to GeoJSON
isochrones_gdf.to_file('isochrones.geojson', driver='GeoJSON')

print("Isochrones GeoJSON saved as 'isochrones.geojson'")